In [55]:
# need to run in the conda --power environment: source activate power; source deactivate.
# functions 
from multiprocessing import Pool
import os
import pickle

import numpy as np
import pandas as pd
import scipy
import skbio

from machivellian.power import subsample_power

In [56]:
overwrite = False

In [57]:
# file handing parameters
txt_delim = '\t'
map_index = '#SampleID'
map_nas = ['NA', 'no_data', 'unknown', '']
write_na = ''

In [58]:
# Loads the sample data into the notebook
all_map = pd.read_csv('7739_analysis_mapping.txt',
                      sep = txt_delim, 
                      na_values = map_nas
                      ) 

all_uud = skbio.DistanceMatrix.read('unweighted_unifrac_dm.txt')

In [59]:
# subset the sample with ids only in unifrac distance
ids = pd.read_csv('unweighted_unifrac_dm.txt',
                      sep = txt_delim, 
                      usecols = [0],
                      na_values = map_nas
                      )
ids.columns = [map_index]
data_map = pd.merge(ids, all_map,  on=[map_index], how='inner')

In [60]:
# subset the sample in nose and foot
data_map = data_map.loc[data_map['body_site'].isin(['UBERON:nose', 'UBERON:zone of skin of foot'])]

In [111]:
# choose the values of parameters
depth = 49

In [11]:
# Justin's version
def test(ids): 
        obs = np.hstack(ids) 
        res = skbio.stats.distance.permanova(
            distance_matrix=dm.filter(obs),
            grouping=groups.loc[obs],
            permutations=depth,
        )
        return res['p-value']
    
# groups = data_map?
# groups.loc[obs] doesn't work

In [112]:
# my modification
def test(ids): 
        obs = np.hstack(ids) 
        res = skbio.stats.distance.permanova(
            distance_matrix=dm.filter(obs),
            grouping=groups,
            permutations=depth,
        )
        return res['p-value']
    

In [113]:
dm = all_uud
groups = data_map['body_site']
ids2 = data_map[map_index]
test(ids2)

0.02

In [114]:
ids2

3      449.F33Fotr
5      449.F32Fotl
7      449.M24Fotl
23     449.M53Fotr
38     449.F21Fotl
65     449.M53Fotl
67     449.F31Fotr
68     449.F32Nose
96     449.F34Fotr
97     449.M64Fotl
108    449.M54Fotr
117    449.M31Fotr
121    449.M32Nose
139    449.M41Nose
142    449.M22Nose
151    449.M42Nose
169    449.M23Fotr
170    449.F33Fotl
171    449.F34Fotl
172    449.M11Fotl
173    449.M22Fotr
175    449.M21Fotl
176    449.M21Fotr
178    449.F14Fotr
179    449.F13Fotr
180    449.F12Fotl
183    449.M32Fotr
206    449.F12Nose
220    449.M22Fotl
227    449.F31Fotl
243    449.M64Fotr
244    449.M14Fotr
251    449.M31Nose
269    449.M44Fotl
281    449.M43Fotr
284    449.F32Fotr
287    449.M14Fotl
290    449.F22Nose
296    449.M12Fotl
311    449.F24Fotr
312    449.F12Fotr
343    449.M12Nose
346    449.F21Nose
357    449.F31Nose
366    449.F22Fotl
383    449.M43Fotl
386    449.F22Fotr
391    449.F23Fotr
395    449.M41Fotr
396    449.M41Fotl
397    449.M42Fotl
400    449.F24Fotl
408    449.F

In [108]:
# set values for parameters
samples = np.vstack(ids2)
alpha = 0.1
num_runs = 2
num_iter = 10
counts = np.arange(2, 5, 1)

In [109]:
counts

array([2, 3, 4])

In [110]:
 # Calculates power
power = subsample_power(test=test,
                        samples=samples,
                        counts=counts,
                        num_iter=100,
                        num_runs=5,
                        alpha=alpha,
                        bootstrap=False,
                        )

ValueError: Sampling depth is too high. Please use replacement or pick fewer observations.